In [ ]:
import numpy as np
import cupy as cp
import h5py
from holotomocupy.holo import G, GT
from holotomocupy.shift import S, ST
from holotomocupy.recon_methods import multiPaganin
from holotomocupy.utils import *
from holotomocupy.proc import remove_outliers
##!jupyter nbconvert --to script config_template.ipynb

# Init data sizes and parametes of the PXM of ID16A

In [ ]:
n = 512  # object size in each dimension
energy = 33.35  # [keV] xray energy
wavelength = 1.2398419840550367e-09/energy  # [m] wave length
focusToDetectorDistance = 1.28  # [m]
ndist = 4
distances = np.array([0.0029432,0.00306911,0.00357247,0.00461673])[:ndist] # [m]
magnification = 400
detector_pixelsize = 3.03751e-6
voxelsize = detector_pixelsize/magnification*2048/n  # object voxel size

ne = n+n//4
pad=n//8
show = True

path = f'/data/vnikitin/modeling/siemens{n}'
fs=n//16

## Read data

In [ ]:
data = np.load(f'{path}/data.npy')
psi = np.load(f'{path}/psi.npy')
rdata = data.copy()

# Construct operators


In [ ]:
def Lop(psi):
    data = cp.zeros([1,ndist, n, n], dtype='complex64')
    for i in range(ndist):
        psir = cp.array(psi)           

        # v = cp.ones(ne,dtype='float32')
        # v[:fs] = cp.sin(cp.linspace(0,1,fs)*cp.pi/2)
        # v[-fs:] = cp.cos(cp.linspace(0,1,fs)*cp.pi/2)
        # v = cp.outer(v,v)


        # psir*=v
        psir = G(psir, wavelength, voxelsize, distances[i],'constant')        

        data[:, i] = psir[:,ne//2-n//2:ne//2+n//2,ne//2-n//2:ne//2+n//2]
    return data

def LTop(data):
    psi = cp.zeros([1, ne, ne], dtype='complex64')
    for j in range(ndist):
        datar = cp.array(cp.pad(data[:, j],((0,0),(ne//2-n//2,ne//2-n//2),(ne//2-n//2,ne//2-n//2)))).astype('complex64')        
        datar = GT(datar, wavelength, voxelsize, distances[j],'constant')        

        # v = cp.ones(ne,dtype='float32')
        # v[:fs] = cp.sin(cp.linspace(0,1,fs)*cp.pi/2)
        # v[-fs:] = cp.cos(cp.linspace(0,1,fs)*cp.pi/2)
        # v = cp.outer(v,v)
        
        # datar *= v
        psi += datar
    return psi

# adjoint tests
tmp = cp.array(data).copy().astype('complex64')
arr1 = cp.pad(tmp[:,0],((0,0),(ne//2-n//2,ne//2-n//2),(ne//2-n//2,ne//2-n//2)),'symmetric')     

arr2 = Lop(arr1)
arr3 = LTop(arr2)


print(f'{np.sum(arr1*np.conj(arr3))}==\n{np.sum(arr2*np.conj(arr2))}')


## Reconstruction with the CG (Carlsson) with Hessians

## Gradients

#### $$\nabla F=2 \left(L^*\left( L(\psi_0)-\tilde D\right)\right).$$
#### where $$\tilde D = (D) \frac{L(\psi_0)}{|L(\psi_0)|}$$



In [ ]:
def gradientF(vars,d):
    psi = vars['psi']
    Lpsi = Lop(psi)
    td = d*(Lpsi/np.abs(Lpsi))
    res = 2*LTop(Lpsi - td)
    return res



##### $$\frac{1}{2}\mathcal{H}|_{x_0}(y,z)= \left\langle \mathbf{1}-d_{0}, \mathsf{Re}({L(y)}\overline{L(z)})\right\rangle+\left\langle d_{0},(\mathsf{Re} (\overline{l_0}\cdot L(y)))\cdot (\mathsf{Re} (\overline{l_0}\cdot L(z))) \right\rangle$$
##### 
##### $$l_0=L(x_0)/|L(x_0)|$$
##### $$d_0=(d)/|L(x_0)|$$


In [ ]:
def hessianF(hpsi,hpsi1,hpsi2,data):
    Lpsi = Lop(hpsi)        
    Lpsi1 = Lop(hpsi1)
    Lpsi2 = Lop(hpsi2)    
    l0 = Lpsi/np.abs(Lpsi)
    d0 = (data)/np.abs(Lpsi)
    v1 = np.sum((1-d0)*reprod(Lpsi1,Lpsi2))
    v2 = np.sum(d0*reprod(l0,Lpsi1)*reprod(l0,Lpsi2))        
    
    
    return 2*(v1+v2)

In [ ]:
def calc_beta(vars,grads,etas,d):
    psi = vars['psi']
    dpsi1 = grads['psi']
    dpsi2 = etas['psi']    
    top = hessianF(psi, dpsi1, dpsi2, d)    
    bottom = hessianF(psi, dpsi2, dpsi2,d)
    return top/bottom

def calc_alpha(vars,grads,etas,d):    
    psi = vars['psi']
    dpsi1 = grads['psi']
    dpsi2 = etas['psi']    
    top = -redot(dpsi1,dpsi2)    
    bottom = hessianF(psi, dpsi2, dpsi2,d)
    return top/bottom, top, bottom

### Initial guess for reconstruction (Paganin)

In [ ]:
def rec_init(rdata):
    recMultiPaganin = cp.zeros([1,ndist,n,n],dtype='float32')
    for j in range(0,ndist):
        rdatar = cp.array(rdata[:,j:j+1])
        r = multiPaganin(rdatar,
                            distances[j:j+1], wavelength, voxelsize,  24.05, 1.2e-6)    
        recMultiPaganin[:,j] = r
        
    recMultiPaganin = np.sum(recMultiPaganin,axis=1)/ndist
    
    recMultiPaganin = np.exp(1j*recMultiPaganin)

    return recMultiPaganin

rec_paganin = rec_init(rdata)
rec_paganin = np.pad(rec_paganin,((0,0),(ne//2-n//2,ne//2-n//2),(ne//2-n//2,ne//2-n//2)),'constant',constant_values=1)
mshow_polar(rec_paganin[0],show)
mshow_polar(rec_paganin[0,ne//2-128:ne//2+128,ne//2-128:ne//2+128],show)

## debug functions

In [ ]:
def plot_debug2(vars,etas,top,bottom,alpha,data):
    if show==False:
        return
    psi = vars['psi']
    dpsi2 = etas['psi']    
    npp = 17
    errt = cp.zeros(npp*2)
    errt2 = cp.zeros(npp*2)
    for k in range(0,npp*2):
        psit = psi+(alpha*k/(npp-1))*dpsi2
        fpsit = np.abs(Lop(psit))-data
        errt[k] = np.linalg.norm(fpsit)**2
        
    t = alpha*(cp.arange(2*npp))/(npp-1)
    tmp = np.abs(Lop(psi))-(data)
    errt2 = np.linalg.norm(tmp)**2-top*t+0.5*bottom*t**2
    
    plt.plot(alpha.get()*cp.arange(2*npp).get()/(npp-1),errt.get(),'.')
    plt.plot(alpha.get()*cp.arange(2*npp).get()/(npp-1),errt2.get(),'.')
    plt.show()


def vis_debug(vars,data,i):
    mshow_polar(vars['psi'][0],show)    
    mshow_polar(vars['psi'][0,ne//2-n//4:ne//2+n//4,ne//2+n//4:ne//2+n//2+n//4],show)    
    
def err_debug(vars, grads, data):    
    psi = vars['psi']
    tmp = np.abs(Lop(psi))-data
    err = np.linalg.norm(tmp)**2
    return err

# Main CG loop (fifth rule)

In [ ]:
# iter = 288
# prb_abs = dxchange.read_tiff(f'/data2/vnikitin/nfp_codes_correct/crec_prb_abs2048_-0.01775full0.3/{iter}.tiff')
# prb_angle = dxchange.read_tiff(f'/data2/vnikitin/nfp_codes_correct/crec_prb_angle2048_-0.01775full0.3/{iter}.tiff')
# prb = prb_abs*np.exp(1j*prb_angle)[np.newaxis]
# prb = prb[:,prb.shape[1]//2-n//2-pad:prb.shape[1]//2+n//2+pad,prb.shape[2]//2-n//2-pad:prb.shape[2]//2+n//2+pad]
# # prb = np.pad(prb,((0,0),(n//2+pad-prb.shape[1]//2,(n//2+pad-prb.shape[1]//2)),(n//2+pad-prb.shape[1]//2,(n//2+pad-prb.shape[1]//2))),'edge')

# mshow_polar(prb[0],show)
# z1c = -17.75e-3
# distances2 = (z1-z1c)/(z1c/z1)#magnifications2
# prb = G(prb,wavelength,voxelsize,distances2[0],'symmetric')
# mshow_polar(prb[0],show)



In [ ]:
def cg_holo(data, vars, pars):

    data = np.sqrt(data)    
    erra = cp.zeros(pars['niter'])
    alphaa = cp.zeros(pars['niter'])    
    grads ={}
    for i in range(pars['niter']):          
        if i % pars['vis_step'] == 0 and pars['vis_step'] != -1:
            vis_debug(vars, data, i) 
        grads['psi'] = gradientF(vars,data)        
        if i==0:
            etas = {}
            etas['psi'] = -grads['psi']
        else:      
            beta = calc_beta(vars, grads, etas, data)
            etas['psi'] = -grads['psi'] + beta*etas['psi']
        alpha,top,bottom = calc_alpha(vars, grads, etas, data) 
        if i % pars['vis_step'] == 0:
            plot_debug2(vars,etas,top,bottom,alpha,data)

        vars['psi'] += alpha*etas['psi']
        
        if i % pars['err_step'] == 0:
            err = err_debug(vars, grads, data)    
            print(f'{i}) {alpha=:.5f}, {err=:1.5e}',flush=True)
            erra[i] = err
            alphaa[i] = alpha
                
    return vars,erra,alphaa

vars = {}
vars['psi'] = cp.array(rec_paganin).copy()
data_rec = cp.array(data)#cp.pad(cp.array(data),((0,0),(0,0),(pad,pad),(pad,pad)))

pars = {'niter': 1025, 'err_step':32, 'vis_step': 128}
vars,erra,alphaa = cg_holo(data_rec, vars, pars)    

In [ ]:
plt.plot(np.arange(0,10000,128),erra[::128].get())
plt.yscale('log')